In [1]:
import numpy as np
import pandas as pd
import random
from Bio import motifs
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from sklearn.cluster import KMeans
from msresist.pre_processing import preprocessing
import warnings
warnings.simplefilter("ignore")

In [2]:
pd.set_option('display.max_colwidth', 1000)

ABC = preprocessing(motifs=True, Vfilter=True, FCfilter=True, log2T=True)
ABC = ABC[~ABC["peptide-phosphosite"].str.contains("-")]

header = ABC.columns
treatments = ABC.columns[2:12]

data = ABC.iloc[:,2:12].T
protnames = list(ABC.iloc[:,1])
Allseqs = list(ABC.iloc[:,0])

Amino acid frequency:

In [3]:
AAfreq = {"A":0.074, "R":0.042, "N":0.044, "D":0.059, "C":0.033, "Q":0.058, "E":0.037, "G":0.074, "H":0.029, "I":0.038, "L":0.076, "K":0.072, "M":0.018, "F":0.04, "P":0.05, "S":0.081, "T":0.062, "W":0.013, "Y":0.033, "V":0.068}

Define clusters by k-means:

In [4]:
kmeans = KMeans(4).fit(data.T)
X = ABC.assign(cluster=kmeans.labels_)

seqs = []
for i in range(0, max(kmeans.labels_) + 1):
    seqs.append(list(X[X["cluster"] == i].iloc[:, 0]))

Generate Seq Instances:

In [5]:
instances = []
for i in range(len(seqs)):
    currentcl = []
    for seq in seqs[i]:
        currentcl.append(Seq(seq.upper(), IUPAC.protein))
    instances.append(currentcl)

Create Motif objects for each cluster, build a PSSM for each cluster and print the information content of the motif compared to the background (relative entropy):

In [6]:
for i in range(len(instances)):
    m = motifs.create(instances[i])
#     m.weblogo("cluster %0.f motif.png" % (i+1))
    pwm = m.counts.normalize(pseudocounts=AAfreq)
    pssm = pwm.log_odds()
    print("cluster %0.f: consensus motif = %s, mean = %0.2f, standard deviation = %0.2f, max = %0.2f, min = %0.2f" % (i+1, m.consensus, pssm.mean(), pssm.std(), pssm.max, pssm.min))

cluster 1: consensus motif = SSSPPYVSLRG, mean = 6.84, standard deviation = 2.84, max = 16.54, min = -80.80
cluster 2: consensus motif = AEDDRYDEERD, mean = 7.66, standard deviation = 3.11, max = 18.71, min = -87.40
cluster 3: consensus motif = KSKGEYDVLVP, mean = 9.12, standard deviation = 3.34, max = 21.05, min = -84.97
cluster 4: consensus motif = SKEEKYGTVRS, mean = 8.03, standard deviation = 3.14, max = 18.97, min = -81.68


Re-implement using all sequences instead of clusters:

In [7]:
instances_ = []
for seq in Allseqs:
    instances_.append(Seq(seq.upper(), IUPAC.protein))

m_ = motifs.create(instances_)
# m_.weblogo("allseqs_motif.png")
pwm_ = m_.counts.normalize(pseudocounts=AAfreq)
pssm_ = pwm_.log_odds()
print("All sequences: consensus motif = %s, mean = %0.2f, standard deviation = %0.2f, max = %0.2f, min = %0.2f" % (m_.consensus, pssm_.mean(), pssm_.std(), pssm_.max, pssm_.min))

All sequences: consensus motif = ASEGRYDTLRE, mean = 6.29, standard deviation = 2.64, max = 15.17, min = -72.46


## Re-implementation from Schwartz & Gygi Nat. Biotech 2005 and Cheng et al Bioinfo. 2018

Build Background data set and position-weight matrix:

In [8]:
bg_seqs = []
# for seq in Allseqs:
#     shuffAA = seq[:5] + seq[6:]
#     shuffled = ''.join(random.sample(shuffAA, 5)) + seq[5] + ''.join(random.sample(shuffAA, 5))
#     shuffled = ''.join(random.sample(seq,11))
#     bg_seqs.append(Seq(shuffled.upper(), IUPAC.protein))

AAlist = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]
for i in range(len(Allseqs)*10):
    seq = ''.join(random.sample(AAlist, 11))
    bg_seqs.append(Seq(seq, IUPAC.protein))

In [9]:
bg_m = motifs.create(bg_seqs)
bg_pwm = pd.DataFrame(bg_m.counts.normalize(pseudocounts=AAfreq)).T

Build Phosphorylation data set and position-weight matrix:

In [10]:
seqs = []
for seq in Allseqs:
    seqs.append(Seq(seq.upper(), IUPAC.protein))

In [11]:
m = motifs.create(seqs)
pwm = pd.DataFrame(m.counts.normalize(pseudocounts=AAfreq)).T

Calculate Binomial Probability Matrix:

In [12]:
from scipy.stats import binom

n = len(seqs)
k = pd.DataFrame(m.counts).T.reset_index(drop=False)
p = bg_pwm

binomp = []
for i, r in k.iterrows():
    CurrentResidue = []
    for j,v in enumerate(r[1:]):
        CurrentResidue.append(binom.sf(k=v, n=n, p=p.iloc[i, j], loc=0))
    binomp.append(CurrentResidue)
    
binomp = pd.DataFrame(binomp)
binomp.insert(0, "Residue", list(k.iloc[:,0]))

In [13]:
binomp

,Residue,0,1,2,3,4,5,6,7,8,9,10
0,A,0.000145,0.108945,0.233556,0.000875,1.624863e-01,1.000000e+00,1.050134e-02,0.514419,9.621395e-01,0.099835,0.008575
1,C,0.999766,0.998881,0.999997,0.999992,1.000000e+00,9.999999e-01,1.000000e+00,0.999989,1.000000e+00,0.999975,0.999630
2,D,0.155414,0.029403,0.002730,0.001076,2.579478e-03,1.000000e+00,7.493248e-10,0.440902,8.705733e-01,0.946737,0.013034
3,E,0.181105,0.011314,0.000003,0.012117,1.233911e-03,1.000000e+00,2.061929e-03,0.000365,4.156734e-02,0.063410,0.000188
4,F,0.896026,0.999032,0.993581,0.999368,9.978837e-01,1.000000e+00,9.800052e-01,0.819541,9.305375e-01,0.909448,0.991797
5,G,0.004018,0.088207,0.000534,0.000300,4.616782e-02,1.000000e+00,1.922217e-02,0.390076,3.560138e-01,0.573847,0.000944
6,H,0.998507,0.999905,0.977761,0.916905,9.958543e-01,1.000000e+00,3.151363e-01,0.989553,9.334702e-01,0.966414,0.952451
7,I,0.866015,0.878540,0.998931,0.883114,3.194570e-01,1.000000e+00,3.832656e-01,0.660222,2.847438e-01,0.969682,0.972649
8,K,0.022323,0.000623,0.286364,0.933595,9.224314e-02,1.000000e+00,9.095722e-01,0.748993,2.476566e-01,0.690392,0.020452
9,L,0.015878,0.319982,0.465816,0.015996,1.417096e-03,1.000000e+00,6.408505e-01,0.001238,8.840495e-10,0.144543,0.208204


In [14]:
motif = list("X"*11)
positions = list(binomp.columns[1:])
AA = list(binomp.iloc[:, 0])
binomp = binomp.iloc[:, 1:]
k = k.iloc[:, 1:]
pvalCut = 10**(-6)
occurCut = 20

In [15]:
for i in range(len(positions)):
    DoS = binomp.iloc[:, i].min()
    j = binomp[binomp.iloc[:, i] == DoS].index[0]
    aa = AA[j]
    if DoS < pvalCut and k.iloc[j, i] >= occurCut:
        motif[i] = aa
    else:
        motif[i] = "x"

motif1 = ''.join(motif)

In [16]:
print(motif1)
print(m.consensus)

xxxxRYDxLxx
ASEGRYDTLRE
